In [2]:
import urllib
urllib.urlretrieve ("http://imag.pub.ro/pandora/Download/Pandora_18k.zip", "Pandora_18k.zip")

('Pandora_18k.zip', <httplib.HTTPMessage instance at 0x7f9e9a4393f8>)

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('Pandora_18k.zip', 'r')
zip_ref.extractall("./Pandora_18k")
zip_ref.close()

In [34]:
!ls -alh
#!zip try1_weights.h5.zip try1_weights.h5

total 1.9G
drwxr-xr-x  1 root root 4.0K Apr 25 04:41 .
drwxr-xr-x  1 root root 4.0K Apr 25 01:41 ..
drwx------  4 root root 4.0K Apr 25 01:46 .cache
drwxr-xr-x  3 root root 4.0K Apr 25 01:46 .config
-rw-r--r--  1 root root 159M Apr 25 01:55 CustomVGG.h5
drwxr-xr-x  1 root root 4.0K Apr 25 04:41 datalab
drwxr-xr-x  4 root root 4.0K Apr 25 01:42 .forever
drwxr-xr-x  2 root root 4.0K Apr 25 04:49 .gsutil
drwxr-xr-x  5 root root 4.0K Apr 25 01:46 .ipython
drwxr-xr-x  2 root root 4.0K Apr 25 01:57 .jupyter
drwxr-xr-x  3 root root 4.0K Apr 25 01:55 .keras
drwx------  3 root root 4.0K Apr 25 01:42 .local
drwx------  3 root root 4.0K Apr 25 01:55 .nv
drwxr-xr-x 20 root root 4.0K Apr 25 01:52 Pandora_18k
drwxr-xr-x 20 root root 4.0K Apr 25 01:52 Pandora_18k_merged
-rw-r--r--  1 root root 1.4G Apr 25 01:51 Pandora_18k.zip
-rw-------  1 root root 1.0K Apr 25 01:42 .rnd
-rw-r--r--  1 root root 7.6M Apr 25 04:05 try1-dataset.sav
-rw-r--r--  1 root root 159M Apr 25 03:58 try1_weights.h5
-rw-r--r--  

In [0]:
datapath = './Pandora_18k'
mergedDatapath = './Pandora_18k_merged'
os.system("mkdir "+mergedDatapath)
def preprocess1():
    d = datapath
    dirsNames = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
    dirs = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
    #print dirs

    # move all images from subfolders to a single folder. Ignore some naming conflicts
    for i in xrange(len(dirs)):
        mergedPath = mergedDatapath + '/' + dirsNames[i] + "_merged"
        os.system("mkdir "+mergedPath)
        os.system("find "+dirs[i]+"  -iname '*.jpg' -exec cp -f -t "+mergedPath+"  '{}' +")
        #if i > 1: break
preprocess1()

In [14]:
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

import keras
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
#from baseline import *

mergedDatapath = './Pandora_18k_merged'
img_width,img_height = 224,224

def createCustomModel(num_classes=18):
    VGG_model_path = "VGG16_original.h5"
    #model = VGG16(input_shape = (img_width, img_height, 3))
    #model = load_model(VGG_model_path)
    model = VGG16()
    """
    image = load_img('239.jpg', target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # predict the probability across all output classes
    yhat = model.predict(image)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))
    """

    #model.layers.pop()
    x = model.layers[-5].output # dis regard the fc layers after this
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    # creating the final model 
    model_final = Model(input = model.input, output = predictions)
        #Total 24 layers,16 with weights
    """
    [<keras.engine.topology.InputLayer at 0x7f0eb73f48d0>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb73f4a90>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb73f4ad0>,
     <keras.layers.pooling.MaxPooling2D at 0x7f0eb73f4b50>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb73f4e90>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb73f4ed0>,
     <keras.layers.pooling.MaxPooling2D at 0x7f0eb73f4fd0>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e150>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e190>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e290>,
     <keras.layers.pooling.MaxPooling2D at 0x7f0eb639e390>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e4d0>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e510>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e610>,
     <keras.layers.pooling.MaxPooling2D at 0x7f0eb639e710>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e850>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e890>,
     <keras.layers.convolutional.Conv2D at 0x7f0eb639e990>,
     <keras.layers.pooling.MaxPooling2D at 0x7f0eb639ea90>,  #19 till here freezing
     <keras.layers.core.Flatten at 0x7f0eb4e3c3d0>,
     <keras.layers.core.Dense at 0x7f0eb4e3c2d0>,
     <keras.layers.core.Dropout at 0x7f0eb4e3c050>,
     <keras.layers.core.Dense at 0x7f0eb4e64410>,
     <keras.layers.core.Dense at 0x7f0eb4e64390>]
    """
    for layer in model.layers[:19]:
        layer.trainable = False
    # compile the model 
    model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


    print model_final.summary()
    model_final.save("./CustomVGG.h5")
    return model_final




def getDataset(perClassCount = -1, size=(500,500)):
    
    pkfilename =  './' + 'feature_images_keras.sav'
    
    d = mergedDatapath
    dirsNames = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
    dirs = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

    np.random.seed(0)
    X,Y= [],[]
    for i in range(len(dirs)):
        d = dirs[i]
        filenames = glob.glob(d+'/*.jpg')
        l = len(filenames)

        if perClassCount == -1 : # take all data
            X += filenames
            Y += [i for j in xrange(len(filenames))]
        else:
            f = list(np.random.choice(filenames,replace=False,size=np.min(perClassCount,len(filenames))))
            X += f
            Y += [i for j in xrange(perClassCount)] # label of these images
        

    print("Reading images...")
    images = []
    j = 0
    for i in range(len(X)):
        images.append(img_to_array(load_img(X[i], target_size=size)))
        if len(images) > 17000 : break
        if (i+1)%500==0:
            print "Read ",i+1," images."
    print "Done reading images..."
    images=np.array(images) 
    images=preprocess_input(images)
    print "Done processing images..."
    X = images
    print "Dumping the features for later use.."
    
    Y= keras.utils.to_categorical(np.array(Y),18)
    pickle.dump((X,Y), open(pkfilename, 'wb'))

    return X,Y

def getDataset2():
    '''returns only the files name and not the actual images'''
    d = mergedDatapath
    dirsNames = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
    dirs = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

    X,Y= [],[]
    for i in range(len(dirs)):
        d = dirs[i]
        filenames = glob.glob(d+'/*.jpg')
        l = len(filenames)

        X += filenames
        Y += [i for j in xrange(len(filenames))]
    
    l = len(X)
    X= np.array(X)
    Y=np.array(Y)
    return train_test_split(X,Y,test_size=0.2)
'''
class DataGenerator(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(imread(file_name), (200, 200))
               for file_name in batch_x]), np.array(batch_y)
'''

def DataGenerator(X,Y,size=(224,224),batch_size=20):
    
    

    while 1:

      idxs = np.random.choice(X.shape[0],replace=False,size=batch_size)
      images = []
      for i in idxs:
        images.append(img_to_array(load_img(X[i], target_size=size)))
      images=np.array(images) 
      images=preprocess_input(images)
      yield images,keras.utils.to_categorical(np.array(Y[idxs]),18) # yield image and label pair

       
        

if __name__ == '__main__':
    trainx,testx, trainy ,testy = getDataset2()
    print trainx.shape, testx.shape
  
    #model = createCustomModel()
    model = load_model("./CustomVGG.h5")
    #print model.summary()
    
    """
    X,Y = getDataset(perClassCount = -1,size=(img_width, img_height))
    trainx,testx, trainy ,testy = train_test_split(X,Y,test_size=0.2)
    print trainx.shape, testx.shape
    """
    
    print "Traing the custom model"
    batch = 20
    #model.fit(trainx, trainy,batch_size=batch,nb_epoch=20,shuffle=True,verbose=1,validation_data=(testx, testy))
    model.fit_generator(DataGenerator(trainx,trainy,batch_size=batch),epochs=20,steps_per_epoch=650,verbose=1,validation_data=DataGenerator(testx,testy,batch_size=batch),validation_steps=176)
    model.save_weights("./try1_weights.h5")
    

(14278,) (3570,)
Traing the custom model
Epoch 1/20
650/650 [==============================] - 215s 331ms/step - loss: 6.4100 - acc: 0.2970 - val_loss: 2.4420 - val_acc: 0.3256
Epoch 2/20
 49/650 [=>............................] - ETA: 2:38 - loss: 2.2343 - acc: 0.4214650/650 [==============================] - 214s 330ms/step - loss: 1.8822 - acc: 0.4719 - val_loss: 2.0172 - val_acc: 0.3736
Epoch 3/20
236/650 [=========>....................] - ETA: 1:49 - loss: 1.5059 - acc: 0.5447650/650 [==============================] - 214s 330ms/step - loss: 1.4020 - acc: 0.5771 - val_loss: 1.9332 - val_acc: 0.3915
Epoch 4/20
306/650 [=============>................] - ETA: 1:30 - loss: 1.1604 - acc: 0.6456650/650 [==============================] - 215s 330ms/step - loss: 1.0794 - acc: 0.6680 - val_loss: 1.9102 - val_acc: 0.4173
Epoch 5/20
332/650 [==============>...............] - ETA: 1:24 - loss: 0.9334 - acc: 0.7047650/650 [==============================] - 215s 331ms/step - loss: 0.8909 - acc:

In [36]:
from google.colab import files
import uuid
from google.colab import auth
#pickle.dump((trainx,testx, trainy ,testy), open("try1-dataset.sav", 'wb'))
#files.download("try1-dataset.sav")
#files.download("try1_weights.h5.zip")

def fileUploadToGCS(filepath):
  auth.authenticate_user()
  project_id = 'put your project id here'
  !gcloud config set project {project_id}

  # Make a unique bucket to which we'll upload the file.
  # (GCS buckets are part of a single global namespace.)
  bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

  # Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
  !gsutil mb gs://{bucket_name}
  # Copy the file to our new bucket.
  # Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
  !gsutil cp try1_weights.h5 gs://{bucket_name}/
fileUploadToGCS("./try1_weights.h5")

Updated property [core/project].
Creating gs://colab-sample-bucket-f9cdd672-4844-11e8-be26-0242ac110002/...
Copying file://try1_weights.h5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/158.3 MiB.                                    
